### Dependencies

In [1]:
import os
import re
import shutil

import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

### Path config and read the dataset

In [2]:
img_dir = '/Users/home/HAM10000/ML-836-project/skin-disease-classifier-863-project-rahul-collaboration/Dataset/HAM10000_images_part_2/'

files = os.listdir(img_dir)
print("Files in directory:", files)

Files in directory: ['ISIC_0030858.jpg', 'ISIC_0030680.jpg', 'ISIC_0033389.jpg', 'ISIC_0032097.jpg', 'ISIC_0032929.jpg', 'ISIC_0026784.jpg', 'ISIC_0028971.jpg', 'ISIC_0026948.jpg', 'ISIC_0026790.jpg', 'ISIC_0028965.jpg', 'ISIC_0025299.jpg', 'ISIC_0032083.jpg', 'ISIC_0024839.jpg', 'ISIC_0030694.jpg', 'ISIC_0024811.jpg', 'ISIC_0030864.jpg', 'ISIC_0026960.jpg', 'ISIC_0028795.jpg', 'ISIC_0032915.jpg', 'ISIC_0032901.jpg', 'ISIC_0026974.jpg', 'ISIC_0028781.jpg', 'ISIC_0028959.jpg', 'ISIC_0030870.jpg', 'ISIC_0024805.jpg', 'ISIC_0032054.jpg', 'ISIC_0025528.jpg', 'ISIC_0024636.jpg', 'ISIC_0030643.jpg', 'ISIC_0026021.jpg', 'ISIC_0029312.jpg', 'ISIC_0026747.jpg', 'ISIC_0029474.jpg', 'ISIC_0027459.jpg', 'ISIC_0030125.jpg', 'ISIC_0032732.jpg', 'ISIC_0033438.jpg', 'ISIC_0032726.jpg', 'ISIC_0026753.jpg', 'ISIC_0029460.jpg', 'ISIC_0030131.jpg', 'ISIC_0030657.jpg', 'ISIC_0028018.jpg', 'ISIC_0026035.jpg', 'ISIC_0029306.jpg', 'ISIC_0031549.jpg', 'ISIC_0032040.jpg', 'ISIC_0024622.jpg', 'ISIC_0028030.jpg',

In [ ]:
source_dir = '/Users/home/HAM10000/ML-836-project/skin-disease-classifier-863-project-rahul-collaboration/Dataset/HAM10000_images_part_1/'
destination_dir = '/Users/home/HAM10000/ML-836-project/skin-disease-classifier-863-project-rahul-collaboration/Dataset/HAM10000_images_part_2'

def copy_images(source_dir, destination_dir):
    for filename in os.listdir(source_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):  # Ensure only image files are considered
            source_file = os.path.join(source_dir, filename)
            destination_file = os.path.join(destination_dir, filename)

            # Check if file already exists in destination
            if os.path.exists(destination_file):
                base, extension = os.path.splitext(filename)
                i = 1
                # Generate a new filename by appending a number
                while os.path.exists(os.path.join(destination_dir, f"{base}_{i}{extension}")):
                    i += 1
                destination_file = os.path.join(destination_dir, f"{base}_{i}{extension}")

            # Copy file from source to destination
            shutil.copy2(source_file, destination_file)
            print(f"Copied '{source_file}' to '{destination_file}'")

# Example usage:
copy_images(source_dir, destination_dir)

In [3]:
metadata_file = '/Users/home/HAM10000/ML-836-project/skin-disease-classifier-863-project-rahul-collaboration/Dataset/Metadata/HAM10000_metadata'

df = pd.read_csv(metadata_file)
df.head(10)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear,vidir_modern
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face,vidir_modern
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face,vidir_modern
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back,vidir_modern
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back,vidir_modern


In [4]:
print('Number of Classes -', df['dx'].unique())
print('Number of Classes -', df['dx_type'].unique())

Number of Classes - ['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
Number of Classes - ['histo' 'consensus' 'confocal' 'follow_up']


In [5]:
print('Number of Classes -', df['image_id'].unique())

Number of Classes - ['ISIC_0027419' 'ISIC_0025030' 'ISIC_0026769' ... 'ISIC_0033536'
 'ISIC_0032854' 'ISIC_0032258']


In [6]:
y = df.pop('dx')
y = y.to_frame()
encoder = LabelEncoder()
y = encoder.fit_transform(y)
X = df

/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

print('Training set shape X-', X_train.shape, 'y-', y_train.shape)
print('Training set shape X-', X_test.shape, 'y-', y_test.shape)

Training set shape X- (7511, 7) y- (7511,)
Training set shape X- (2504, 7) y- (2504,)


### Dataset and DataLoader for CNN

In [8]:
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

In [9]:
class SkinLesionsDataset(Dataset):
    def __init__(self, X, y, transform = None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img_name = self.X.iloc[idx]['image_id']
        label = self.y[idx]
        img = Image.open(f'{img_dir}{img_name}.jpg')
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [10]:
training_dataset = SkinLesionsDataset(X_train, y_train, transform=transform)
testing_dataset = SkinLesionsDataset(X_test, y_test, transform=transform)

train_dataloder = DataLoader(training_dataset, batch_size=8)
test_dataloader = DataLoader(testing_dataset, batch_size=8)

In [11]:
data_iterator = iter(train_dataloder)
sample_images, sample_labels = next(data_iterator)

print(sample_labels)
print(sample_images)
print(sample_images.shape)

tensor([5, 5, 5, 5, 4, 5, 5, 5])
tensor([[[[0.9686, 0.9725, 0.9725,  ..., 0.9725, 0.9686, 0.9647],
          [0.9686, 0.9686, 0.9686,  ..., 0.9686, 0.9686, 0.9686],
          [0.9647, 0.9686, 0.9647,  ..., 0.9686, 0.9608, 0.9647],
          ...,
          [0.8745, 0.8745, 0.8353,  ..., 0.8784, 0.8549, 0.8667],
          [0.8627, 0.8784, 0.8431,  ..., 0.8314, 0.8314, 0.8627],
          [0.8667, 0.8784, 0.8627,  ..., 0.8706, 0.8078, 0.8196]],

         [[0.6941, 0.6980, 0.6941,  ..., 0.7020, 0.7059, 0.7020],
          [0.6902, 0.6941, 0.6863,  ..., 0.7059, 0.6980, 0.6941],
          [0.6863, 0.6863, 0.6863,  ..., 0.6941, 0.6824, 0.6902],
          ...,
          [0.6314, 0.6275, 0.5961,  ..., 0.6431, 0.6275, 0.6353],
          [0.6196, 0.6314, 0.5922,  ..., 0.6078, 0.6039, 0.6314],
          [0.6118, 0.6275, 0.6118,  ..., 0.6392, 0.5804, 0.5922]],

         [[0.7294, 0.7333, 0.7333,  ..., 0.7490, 0.7451, 0.7373],
          [0.7255, 0.7294, 0.7255,  ..., 0.7451, 0.7451, 0.7373],
         

### Model Architecture

In [12]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # feature extraction with Convolutions, Relu, and max_pooling layers
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        # classification with linear layers
        self.linear1 = nn.Linear(16*13*13, 128)
        self.linear2 = nn.Linear(128, 64)
        # no softamx activation needed bacause we are using Cross Entropy loss
        # In pytorch, it includes softmax
        self.linear3 = nn.Linear(64, 7) # 7 classes
        

    def forward(self, x):
        # input size = (3,64,64)
        x = nn.functional.relu(self.conv1(x))  # Output size = (6,62,62)
        x = self.pool(x)  # Output size = (6,30,31)
        x = nn.functional.relu(self.conv2(x))  # Output size = (16,26,26)
        x = self.pool(x)  # Output size = (16,13,13)
        x = torch.flatten(x,1)  # Output size = (6,60,60)
        x = nn.functional.relu(self.linear1(x))  # Output size = (6,60,60)
        x = nn.functional.relu(self.linear2(x))  # Output size = (6,60,60)
        x = self.linear3(x)  # Output size = (6,60,60)
        return x

In [13]:
import torch.nn.functional as F

class ImprovedCNNModel(nn.Module):
    def __init__(self):
        super(ImprovedCNNModel, self).__init__()

        # Feature extraction
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # Same padding
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        # Classification
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # Global average pooling
        self.fc1 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 7)  # 7 classes

    def forward(self, x):
        x = F.gelu(self.bn1(self.conv1(x)))
        x = self.pool(F.gelu(self.bn2(self.conv2(x))))
        x = F.gelu(self.bn3(self.conv3(x)))
        x = self.pool2(F.gelu(self.bn4(self.conv4(x))))
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = F.gelu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [14]:
model = ImprovedCNNModel().to('cpu')
print(model)

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

ImprovedCNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (global_avg_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (dropout): Dropout(p

### Training and Validation Loop

In [15]:
def train_model(model, optimizer, loss_func, num_epochs, device, train_dataloader, test_dataloader):
    for current_epoch in range(num_epochs):
        model.train()
        running_loss = 0
        correct_train_predictions = 0
        for _, (batch_images, batch_labels) in enumerate(train_dataloader):
            imgs =  batch_images.to(device)
            labels = torch.tensor(batch_labels).to(device)
            # Forward pass, get the output from the model and
            # calculate the loss by comparing the model output and true labels
            model_output = model(imgs)
            loss = loss_func(model_output, labels)
            
            # Backpropagate the calculated loss
            optimizer.zero_grad() #zeroing the gradients
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predictions = torch.max(model_output, 1)  # Get the higest probability predictions
            correct_train_predictions += (predictions ==labels).sum().item()

        running_loss = running_loss/len(train_dataloader)
        train_accuracy = correct_train_predictions/len(train_dataloader)
        # val_accuracy = 
        print(f'Epoch [{current_epoch}/{num_epochs}] - ')
        print(f'   Training Accuracy: {train_accuracy} Validation Accuracy:  Loss: {running_loss}')

    print('Training Completed!')
        


### Train the Model

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
num_epochs = 5
batch_size = 1
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

training_dataset = SkinLesionsDataset(X_train, y_train, transform=transform)
testing_dataset = SkinLesionsDataset(X_test, y_test, transform=transform)

train_dataloader = DataLoader(training_dataset, batch_size)
test_dataloader = DataLoader(testing_dataset, batch_size)

model = ImprovedCNNModel().to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD([
    {'params': model.parameters(), 'lr': 0.01},
])


train_model(model, optimizer, loss_function, num_epochs, device, train_dataloader, test_dataloader)

/var/folders/4z/8pqfq0zd4b95wqlwjtlwrstr0000gn/T/ipykernel_44219/1319176066.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch_labels).to(device)
[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


Epoch [0/5] - 
   Training Accuracy: 0.668219944082013 Validation Accuracy:  Loss: 1.0361748183406865
Epoch [1/5] - 
   Training Accuracy: 0.679403541472507 Validation Accuracy:  Loss: 0.8853309852186264
Epoch [2/5] - 
   Training Accuracy: 0.6907202769271735 Validation Accuracy:  Loss: 0.8321796205735951
Epoch [3/5] - 
   Training Accuracy: 0.6977765943283185 Validation Accuracy:  Loss: 0.8013288673763821
Epoch [4/5] - 
   Training Accuracy: 0.7114898149380908 Validation Accuracy:  Loss: 0.7692055868808746
Training Completed!


# Transfer Learning

In [25]:
# transfer learning using popular models like resnet, vgg, densenet, etc
# using pretrained models
# fine tuning
# using learning rate scheduler
# using early stopping
# using data augmentation
# using more complex models
# using ensemble methods
# using hyperparameter tuning

# using pretrained models
from torchvision import models

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 7)  # Change the output layer to have 7 classes
model = model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_model(model, optimizer, loss_function, num_epochs, device, train_dataloader, test_dataloader)

/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/home/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/home/HAM10000/ML-836-project/skin-disease-classifier-863-project-rahul-collaboration/Dataset/HAM10000_images_part_2/ISIC_0030055.jpg'

### Saving the Model